In [1]:
f

In [2]:
with open('data/opensubtitles/OpenSubtitles.en-th.en','r', encoding='utf-8') as f:
    en = f.read().split('\n')
len(en),en[:3]

(3281534,
 ['Slave in the Magic Mirror, come from the farthest space.',
  'Through wind and darkness, I summon thee.',
  'Speak!'])

In [3]:
with open('data/opensubtitles/OpenSubtitles.en-th.th','r', encoding='utf-8') as f:
    th = f.read().split('\n')
len(th),th[:3]

(3281534,
 ['ทาสในกระจกวิเศษ, มาจากพื้นที่ที่ไกลที่สุด',
  'ผ่านลมและความมืดฉันเรียกเจ้า',
  'พูด!'])

## 1. subword-nmt

-----

- pre-tokenize the input text (both source and target) with `newmm`

In [4]:
counter = 0
# list_of_unk_toks = [b'\xc2\x99', b'\xc2\x9a', b'\xc2\x95',  b'\xc2\x97', b'\xc2\x98',
#                     b'\xc2\x80', b'\xc2\x81', b'\xc2\x82', b'\xc2\x83', b'\xc2\x84',
#                     b'\xc2\x86',
#                     b'\xc2\x87', b'\xc2\x88', 
#                     b'\xc2\x89', b'\xc2\x8a']
  
list_of_unk_toks = [b'\x98\xc2', b'\xae\xc2', b'\x99\xc2', b'\xb1\xc2' , b'\xc2\xb7', b'\xc3\x83']
# for index, text in enumerate(th):
#     for token in list_of_unk_toks:
#         if token in text.encode('utf-8'):
#             print('found', token)
#             print('index [{}]: {}'.format(index, text))
#             print('English:', en[index])
#             print('---')
#             counter += 1
#             break
            
            
# #     break

# print('total number of lines that contain "เธ..." =', counter)



In [5]:
substring_lists = ['', '', '​', '', '', '', '', '', 'โ', '​',
                   '♪', '{\ cHFFFFFF }', '§', 'font color = "# 808080 "']
for item in substring_lists:
    print(item.encode('utf-8'))


b'\xc2\x99'
b'\xc2\x84'
b'\xe2\x80\x8b'
b'\xc2\x94'
b'\xc2\x96'
b'\xc2\x93'
b'\xc2\x85'
b'\xc2\x97'
b'\xe0\xb9\x82\xc2\x80'
b'\xe2\x80\x8b'
b'\xe2\x99\xaa'
b'{\\ cHFFFFFF }'
b'\xc2\xa7'
b'font color = "# 808080 "'


/usr/local/lib/python3.6/dist-packages/IPython/core/inputtransformer2.py:468: UserWarning: `make_tokens_by_line` received a list of lines which do not have lineending markers ('\n', '\r', '\r\n', '\x0b', '\x0c'), behavior will be unspecified
  warnings.warn("`make_tokens_by_line` received a list of lines which do not have lineending markers ('\\n', '\\r', '\\r\\n', '\\x0b', '\\x0c'), behavior will be unspecified")


In [6]:
def unescape_string(text):
    return html.unescape(text)

print(unescape_string('\\" \' \\'))

\" ' \


In [7]:
_tokenizer_newmm = partial(word_tokenize, engine='newmm', keep_whitespace=False)
def tokenize_worker(sentence, lang):
    
    for substring in substring_lists:
        sentence = sentence.replace(substring, '')
    sentence = unescape_string(sentence)
    return ' '.join(_tokenizer_newmm(sentence))

    
def tokenize_handler(sentences, lang):
    toks = []
    p = Pool(12)
    t = time()
    _tokenize_worker = partial(tokenize_worker, lang=lang)
    toks = p.map(_tokenize_worker, sentences)
    
    p.close()
    p.join() # call Pool.join() to wait for the worker processes to terminate.

    print('{} s'.format(time() -t))

    return toks


def sentences_filter(sentences, lang):
    indices = []
    for index, sentence in tqdm_notebook(enumerate(sentences), total=len(sentences)):
        for token in list_of_unk_toks:
            if token in sentence.encode('utf-8'):
                indices.append(index)
                break
        if len(sentence) <= 1:
            indices.append(index)
            continue
        if lang == 'th' and countthai(sentence, ignore_chars='') == 0.0:
            indices.append(index)
            continue
    return indices



In [8]:
t = time()
indices_to_filter_out_th = sentences_filter(th, lang='th')
indices_to_filter_out_en = sentences_filter(en, lang='en')

print(len(indices_to_filter_out_th))
print(len(indices_to_filter_out_en))

indices_to_filter_out = indices_to_filter_out_th + indices_to_filter_out_en
indices_to_filter_out = set(indices_to_filter_out)

print('{} seconds'.format(time() -t))



76544
4392
58.89504814147949 seconds


In [9]:
# th remove 76544 
# en remove 4392

In [10]:
filtered_th = [x for i, x in enumerate(th) if i not in indices_to_filter_out]
filtered_en = [x for i, x in enumerate(en) if i not in indices_to_filter_out]


In [11]:
print(len(th), len(en))
print(len(filtered_th), len(filtered_en))
print('diff: {} sentences'.format((len(filtered_th) - len(th))))


3281534 3281534
3202751 3202751
diff: -78783 sentences


In [12]:
en_tok = tokenize_handler(filtered_en[:], lang="en")


36.50061297416687 s


In [13]:
print(len(en_tok), en_tok[:5])


3202751 ['Slave in the Magic Mirror , come from the farthest space .', 'Through wind and darkness , I summon thee .', 'Speak !', 'Let me see thy face .', 'What wouldst thou know , my Queen ?']


In [14]:
th_tok = tokenize_handler(filtered_th[:],  lang="th")


36.643035650253296 s


In [15]:
print(len(th_tok), th_tok[:5])


3202751 ['ทาส ใน กระจก วิเศษ , มาจาก พื้นที่ ที่ ไกล ที่สุด', 'ผ่าน ลม และ ความมืด ฉัน เรียก เจ้า', 'พูด !', 'ให้ ฉัน เห็น พระพักตร์ ของ พระองค์', 'สิ่ง ที่ เจ้า จะ รู้ ว่า สมเด็จ พระราชินี ของ ฉัน ได้ อย่างไร']


In [17]:
#train-valid-test split 80/10/10

n = len(th_tok)
print('n=',n)
idx = list(range(n))

random.seed(1234) # Set SEED
random.shuffle(idx)

train_idx, valid_idx, test_idx = idx[:int(n*0.8)], idx[int(n*0.8):int(n*0.9)], idx[int(n*0.9):]

len(train_idx),len(valid_idx),len(test_idx)



n= 3202751


(2562200, 320275, 320276)

In [18]:
en_train = [en_tok[i] for i in train_idx]
print(len(en_train))

en_valid = [en_tok[i] for i in valid_idx]
print(len(en_valid))

en_test = [en_tok[i] for i in test_idx]
print(len(en_test))


2562200
320275
320276


In [19]:
th_train = [th_tok[i] for i in train_idx]
print(len(th_train))

th_valid = [th_tok[i] for i in valid_idx]
print(len(th_valid))

th_test = [th_tok[i] for i in test_idx]
print(len(th_test))


2562200
320275
320276


In [20]:
# folder name is now `opensubtitles_tok_bpe` for this sanbox_bpe.ipynb

FOLDER_NAME = "opensubtitles_tok_bpe"
FOLDER_NAME_BIN = "opensubtitles_bin_bpe"


In [21]:
def write_spaced_tokens_to_file(data, filename):
    with open('./data/{}/{}'.format(FOLDER_NAME, filename),'w') as f:
        for item in data:
            f.write(item + '\n')
            
            

In [22]:
write_spaced_tokens_to_file(en_train, 'train.en')
write_spaced_tokens_to_file(th_train, 'train.th')
write_spaced_tokens_to_file(en_valid, 'valid.en')
write_spaced_tokens_to_file(th_valid, 'valid.th')
write_spaced_tokens_to_file(en_test, 'test.en')
write_spaced_tokens_to_file(th_test, 'test.th')


In [23]:
en_train[:5], th_train[:5]


(['Becky , um , you were acting particularly strange in there just now .',
  "Stay with her so Anna can guide you . I ' m going back .",
  'Look .',
  "Oh , no , it ' s the other way around , Dr . Lewis .",
  'Sort of .'],
 ['เบค กี้ เธอ ทำท่า แปลก ๆ เมื่อกี้ ใน ห้อง',
  'อยู่ กับ เธอ แอน นา จะ นำทาง คุณ ผม จะ กลับ ไป',
  'ฟัง นะ',
  'พอดี เลย ดร. ลี วิ ส',
  'แบบ ว่า'])

In [24]:
en_valid[:5], th_valid[:5]


(['Looking sharp , there , Mozzie .',
  "I knew you ' d come home .",
  'I hide in plain sight , same as you .',
  "Dude , I wanted to cover actual news , not Nicki Minaj ' s vagina fuckin ' flopping out at the Grammys .",
  "Sammy ' s long gone ."],
 ['ดู เฉี่ยว มาก มอ ส ซี่',
  'ฉัน รู้ ว่า เธอ ต้อง กลับบ้าน',
  'ผม ซ่อนตัว อยู่ กับ คน ทั่วไป เช่นเดียวกัน กับ คุณ',
  'เพื่อน เอ๋ย ชั้น อยากได้ พาดหัว ที่ เป็นข่าว จริงๆ ไม่ใช่ จิ๋ม นิ ก กี้ มิ นา จ ที่ แพลม ๆ ออกมา ตอน งาน แกรมมี่',
  'แซม มี่ ไป แล้ว'])

In [25]:
# # en_test[:1000], th_test[:1000]
# for i in range(0,10000):
#     print(en_test[i])
#     print(th_test[i])
#     print()

In [142]:
# test Wh Question
counter = Counter()
WH_questions = ["What", "Why", "When", "Where", "Who", "How"]
for i in range(len(th_tok)):
#     if len(th_tok[i]) <= 2:
    if th_tok[i][-1] == "?" and en_tok[i][-1] == "?" and any(s in en_tok[i] for s in WH_questions):
        if counter['th_contains_question_mark'] < 20:
            print('Thai sentence contains question mark:')
            print('  en:', en_tok[i])
            print('  th:', th_tok[i])
            print('------------\n')

#         print('th:', th_tok[i].encode('utf-8'))
#         print('------------')
        counter.update({'th_contains_question_mark': 1})
    if th_tok[i][-1] != "?" and en_tok[i][-1] == "?" and any(s in en_tok[i] for s in WH_questions):
        if counter['th_not_contain_question_mark'] < 20:
            print('Thai sentence does NOT contain question mark:')
            print('  en:', en_tok[i])
            print('  th:', th_tok[i])
            print('------------\n')
#         print('th:', th_tok[i].encode('utf-8'))
        counter.update({'th_not_contain_question_mark': 1})

print('counter=', counter)

Thai sentence does NOT contain question mark:
  en: What wouldst thou know , my Queen ?
  th: สิ่ง ที่ เจ้า จะ รู้ ว่า สมเด็จ พระราชินี ของ ฉัน ได้ อย่างไร
------------

Thai sentence contains question mark:
  en: What ' s the matter ?
  th: เกิด อะไร ขึ้น ?
------------

Thai sentence does NOT contain question mark:
  en: Where ' s your mama and papa ?
  th: ที่ไหน ของ แม่ และ พ่อ ของ คุณ หรือไม่
------------

Thai sentence contains question mark:
  en: What do you do when things go wrong ?
  th: คุณ จะ ทำ อย่างไร เมื่อ สิ่ง ผิด ไป ?
------------

Thai sentence contains question mark:
  en: -What ' s that ?
  th: นั่น อะไร ?
------------

Thai sentence contains question mark:
  en: -Did you see it ? -How big is it ?
  th: คุณ เห็น มัน ได้ หรือไม่ มัน ใหญ่ แค่ ไหน ?
------------

Thai sentence contains question mark:
  en: What was it doin '?
  th: มัน เป็น สิ่ง ที่ ทำ '?
------------

Thai sentence contains question mark:
  en: -What is it ?
  th: มัน คือ อะไร ?
------------

Thai sen

In [204]:
counter = Counter()
for i in range(len(th_tok)):
  
    if 'Cease fire' in en_tok[i] or 'Cease fire' in th_tok[i]:
        if counter['contains'] < 100:

            print('  en:', en_tok[i])
            print('  th:', th_tok[i])
            print('------------\n')

#         print('th:', th_tok[i].encode('utf-8'))
#         print('------------')
        counter.update({'contains': 1})
print('counter=', counter)

  en: Cease fire ! Cease fire !
  th: หยุด ยิง หยุด ยิง
------------

  en: Cease fire !
  th: หยุด ยิง
------------

  en: - Cease fire !
  th: - ยิง อะไร น่ะ
------------

  en: All right , cease fire ! Cease fire !
  th: หยุด ยิง
------------

  en: Cease fire ! Cease fire !
  th: หยุด ยิง
------------

  en: - Cease fire !
  th: - หยุด ไฟไหม้ !
------------

  en: - Cease fire !
  th: - หยุด ยิง !
------------

  en: Cease fire !
  th: หยุด ยิง !
------------

  en: Cease fire . Cease fire .
  th: หยุด ยิง หยุด ยิง
------------

  en: Cease fire !
  th: หยุด ยิง !
------------

  en: Cease fire !
  th: หยุด ยิง !
------------

  en: Cease fire !
  th: หยุด ยิง !
------------

  en: Cease fire !
  th: หยุด ยิง !
------------

  en: - Cease fire !
  th: - หยุด ยิง
------------

  en: Cease fire on the waterworks .
  th: - ไม่ต้อง มา ร้องไห้
------------

  en: - Cease fire ! - Hands on your head . - Cease fire .
  th: หยุด ยิง เอา มือ ไว้ บน หัว หยุด ยิง
------------

  en: Cease fir

In [147]:
counter = Counter()
for i in range(len(th_tok)):
    if '" ' in th_tok[i]:
        if counter['th_start_with_dounle_quote'] < 20:

            print('  en:', en_tok[i])
            print('  th:', th_tok[i])
            print('------------\n')

#         print('th:', th_tok[i].encode('utf-8'))
#         print('------------')
        counter.update({'th_start_with_dounle_quote': 1})
    if '" ' in en_tok[i]:
        if counter['en_start_with_dounle_quote'] < 20:

            print('  en:', en_tok[i])
            print('  th:', th_tok[i])
            print('------------\n')

#         print('th:', th_tok[i].encode('utf-8'))
#         print('------------')
        counter.update({'en_start_with_dounle_quote': 1})
print('counter=', counter)

  en: Next thing ya know she ' ll be tying ' your beards up in pink ribbons and smelling ' ya up with that stuff called " perfoom ."
  th: สิ่ง ต่อไป ที่ ยา รู้ เธอ จะ ได้รับ การคาด ' ของ คุณ เครา ขึ้น มา ใน ริบบิ้น สีชมพู
------------

  en: " Love ' s First Kiss ."
  th: ความรัก จูบ แรก ของ
------------

  en: Adenoid Hynkel said , " Tomainia was down but today has risen ."
  th: อ ดีน อย เฮ นเคิล กล่าว และ ฉัน แปล
------------

  en: " Democracy is fragrant ." " Liberty is odious ."
  th: " ประชาธิปไตย มัน ช่าง หอม ยวน ใจ "
------------

  en: " Democracy is fragrant ." " Liberty is odious ."
  th: " ประชาธิปไตย มัน ช่าง หอม ยวน ใจ "
------------

  en: " Freedom of speech is objectionable ."
  th: " สิทธิ ใน การ พูด ไร้สาระ สิ้นดี "
------------

  en: " Freedom of speech is objectionable ."
  th: " สิทธิ ใน การ พูด ไร้สาระ สิ้นดี "
------------

  en: " Tomainia has the greatest army ."
  th: " โท ไมเนีย มี กองทัพ ที่ เกรียง ไก ร. "
------------

  en: " Tomainia has the greatest 

### Learn BPE from train


## 1.1 bpe-to-bpe (en -> th) (separated vocab)

In [ ]:
BPE_TOKENS = 25000

# BPE_CODE = './data/{}/code'.format(FOLDER_NAME)
BPE_CODE_JOINT = './data/{}/code.joint'.format(FOLDER_NAME)

TRAIN_EN = './data/{}/train.en'.format(FOLDER_NAME)
TRAIN_TH = './data/{}/train.th'.format(FOLDER_NAME)
VOCAB_PREFIX= './data/{}/vocab'.format(FOLDER_NAME)


print("learn_bpe.py on ${TRAIN}...")
print(BPE_TOKENS, BPE_CODE, TRAIN_EN, TRAIN_TH, VOCAB_PREFIX)


In [ ]:
# run subword-nmt learn-bpe for Joint TH-EN

!subword-nmt learn-joint-bpe-and-vocab --input {TRAIN_TH} {TRAIN_EN} -s {BPE_TOKENS} -o {BPE_CODE_JOINT} --verbose --write-vocabulary {VOCAB_PREFIX}.th {VOCAB_PREFIX}.en


In [ ]:
for lang in ['en', 'th']:
    for file_prefix in ['train', 'valid', 'test']:
        # <train/valid/test>.<en/th>
        file_name = './data/{}/{}.{}'.format(FOLDER_NAME, file_prefix, lang)
        
        # <train/valid/test>.bpe.<en/th>
        file_name_bpe = './data/{}/{}.bpe.{}'.format(FOLDER_NAME, file_prefix, lang)
       
        print('apply BPE to', file_name)

        !subword-nmt apply-bpe -c {BPE_CODE_JOINT} --vocabulary {VOCAB_PREFIX}.{lang} < {file_name} > {file_name_bpe}
        
        


In [ ]:
!fairseq-preprocess --source-lang en --target-lang th \
    --trainpref data/$FOLDER_NAME/train.bpe \
    --validpref data/$FOLDER_NAME/valid.bpe \
    --testpref data/$FOLDER_NAME/test.bpe \
    --destdir data/$FOLDER_NAME_BIN \
    --bpe subword_nmt \
    --joined-dictionary \
    --workers 12
    
# shared dictionary

In [ ]:
# Namespace(alignfile=None, bpe='subword_nmt', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data/opensubtitles_bin_bpe', fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='en', srcdict=None, target_lang='th', task='translation', tbmf_wrapper=False, tensorboard_logdir='', testpref='data/opensubtitles_tok_bpe/test.bpe', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='data/opensubtitles_tok_bpe/train.bpe', user_dir=None, validpref='data/opensubtitles_tok_bpe/valid.bpe', workers=12)
# | [en] Dictionary: 25311 types
# | [en] data/opensubtitles_tok_bpe/train.bpe.en: 2608023 sents, 27457327 tokens, 0.0% replaced by <unk>
# | [en] Dictionary: 25311 types
# | [en] data/opensubtitles_tok_bpe/valid.bpe.en: 326003 sents, 3437093 tokens, 0.000378% replaced by <unk>
# | [en] Dictionary: 25311 types
# | [en] data/opensubtitles_tok_bpe/test.bpe.en: 326003 sents, 3433729 tokens, 0.000146% replaced by <unk>
# | [th] Dictionary: 25311 types
# | [th] data/opensubtitles_tok_bpe/train.bpe.th: 2608023 sents, 22482753 tokens, 0.0% replaced by <unk>
# | [th] Dictionary: 25311 types
# | [th] data/opensubtitles_tok_bpe/valid.bpe.th: 326003 sents, 2811450 tokens, 0.0% replaced by <unk>
# | [th] Dictionary: 25311 types
# | [th] data/opensubtitles_tok_bpe/test.bpe.th: 326003 sents, 2812530 tokens, 0.0% replaced by <unk>
# | Wrote preprocessed data to data/opensubtitles_bin_bpe

In [ ]:
!fairseq-train \
    data/opensubtitles_bin_bpe \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 2048 \
    --bpe subword_nmt \
    --save-dir data/opensubtitles_model/transformers_bpe \
    --tensorboard-logdir data/opensubtitles_model/transformers_bpe/tensorboard_log 

In [ ]:
# fairseq-generate data/opensubtitles_bin_bpe \
#     --path data/opensubtitles_model/transformers_bpe/checkpoint_best.pt \
#     --beam 5 --remove-bpe


# for transformer-word2word-en2th
# fairseq-generate data/opensubtitles_bin \
#     --path checkpoints/checkpoint_best.pt \
#     --beam 5



!fairseq-generate data/opensubtitles_bin_bpe \
    --path data/opensubtitles_model/transformers_bpe/checkpoint_best.pt \
    --remove-bpe \
    --beam 3 \
    --max-tokens 10000 \
    --nbest 3 \
    --results-path data/opensubtitles_model/transformers_bpe




## 2. SentencePiece (Pre-trained)

-----

- pre-tokenize the input text (both source and target) with `newmm`




## Model inference examples

In [3]:
# from Wang -- 2019-06-26_output_Non_Empty_Answer.txt
pattern = r'answer: \[\'(.+)\'\] variable: (\w.+)'
texts = {
    'th': [],
    'en': []
}
with open('./data/wang/2019-06-26_output_Non_Empty_Answer.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
#         print(line)
#         print(line)
        try:
            match_obj = re.search(pattern, line)

            th_text = match_obj[1]
            if th_text[0] == '?':
                th_text = th_text[1:]
            en_text = match_obj[2]

            texts['th'].append(th_text)
            texts['en'].append(en_text)
        except Exception as e:
            continue
    print('number of examples = ', len(texts['th']))


number of examples =  6118


In [4]:
texts['th'][0:20], texts['en'][0:20]

(['เขาปัดมันอีกครั้ง',
  'เขาปัดมัน',
  'เขาฝังใบหน้าของเขาอีกครั้งในมือของเขาแล้วส่งเสียงครวญคราง',
  'เขาเอามือกุมหน้าของเขาชั่วขณะหนึ่ง',
  'เขาเอามือขึ้นกุมหน้าของเขา',
  'เขาเปล่งเสียงหัวเราะออกมาก',
  'เขาเรียกเช็คบิล และจ่ายเงิน',
  'เขาเรียกเช็คบิล',
  'เขาเรียกมันว่าหมวก แม้ว่ามันจะดูเหมือนหม้อมากกว่าก็เถอะ',
  'เขาเรียกพนักงานเสิร์ฟและสั่งอาหารเย็น',
  'เขากลับมาเมื่อตอนกลางวันพร้อมกับชาวสก็อตแลนด์คนนั้น',
  'เขากลับมาอย่างดีและดำเนินชีวิตต่อไป',
  'เขากลับมาหาตอนเช้าตรู่',
  'เขากลับมาที่หน้าต่าง',
  'เขาป่วยเป็นไข้หวัดใหญ่',
  'เขามานี่เมื่อปีก่อนแล้วก็เปิดเผยตัวกับจีน',
  'เขากลับมาบ้านใต้เสื้ออันเปียกชุ่ม',
  'เขาเข้ามาแล้วพูดกับคนเฝ้าประตู',
  'เขาเข้ามาแล้วยืนหายใจหอบ',
  'เขาเข้ามาในห้องพร้อมวางท่ากร่าง'],
 ['He brushed it off once more.',
  'He brushed it off.',
  'He buried his face again in his hands and groaned aloud.',
  'He buried his face for a moment in his hands.',
  'He buried his face in his hands.',
  'He burst into a laugh.',
  'He called for the bill and 

In [6]:
from fairseq import checkpoint_utils, data, options, tasks
from fairseq.sequence_generator import SequenceGenerator



In [440]:

PATH_MODEL_EN_TH = './data/opensubtitles_model/exp002-1/transformer_base_revised/checkpoint_best.pt'


print('| loading model from {}'.format(PATH_MODEL_EN_TH))

state = checkpoint_utils.load_checkpoint_to_cpu(PATH_MODEL_EN_TH)
args = vars(state['args'])
ensemble_en_th, _, _ = checkpoint_utils.load_model_ensemble_and_task([PATH_MODEL_EN_TH], arg_overrides=args, task=None)


models = {
    'th-en': ensemble_th_en,
    'en-th': ensemble_en_th
}
# # print(task)

| loading model from ./data/opensubtitles_model/exp002-1/transformer_base_revised/checkpoint_best.pt
| [en] dictionary: 90000 types
| [th] dictionary: 88848 types
| loading model from ./data/opensubtitles_model/exp003-1/transformer_base/checkpoint_best.pt
| [th] dictionary: 88848 types
| [en] dictionary: 90000 types


In [448]:

th_dict_1 = data.Dictionary()
th_dict_1.add_from_file('./data/opensubtitles_bin_word/dict.th.txt')

en_dict_1 = data.Dictionary()
en_dict_1.add_from_file('./data/opensubtitles_bin_word/dict.en.txt')

BEAM_SIZE = 5

_generator = SequenceGenerator(tgt_dict=th_dict_1, beam_size=BEAM_SIZE)

print('MT EN to TH, Word to Word')
for i, src in enumerate(texts['en'][:300]):
    print(i+1)
    print('text   :',src)
    toks = word_tokenize(src, engine='newmm', keep_whitespace=False)
    print('tokens :', toks)
    src_indices = [en_dict_1.index(t) for t in toks]
    src_len = [len(src_indices)]
  
    src_indices_tensor = torch.LongTensor(src_indices).view(1, -1)
    src_len_tensor = torch.LongTensor(src_len).view(1, -1)
    
#     print('src indices:', src_indices_tensor.size(), src_indices_tensor)
#     print('src len    :', src_len_tensor.size(), src_len_tensor)

    sample = {
        'net_input': {
                'src_tokens': src_indices_tensor, 'src_lengths': src_len_tensor,
        },
    }
    
    hypos = _generator.generate(models=models['en-th'], sample=sample)
    
    print('\nTarget      :', texts['th'][i])
    print('Target toks : {}'.format(' '.join(word_tokenize(texts['th'][i], keep_whitespace=""))))
    print('')
    for i in range(0, BEAM_SIZE):
        hypo = hypos[0][i]
        indices = list(hypo['tokens'].clone().detach().numpy())
        
        out_symbols = ' '.join([th_dict_1[idx] for idx in indices])
        print('Prediction {}: {}'.format(i+1, out_symbols))
        
    print('\n\n')
#     break

MT EN to TH, Word to Word
1
text   : He brushed it off once more.
tokens : ['He', 'brushed', 'it', 'off', 'once', 'more', '.']

Target      : เขาปัดมันอีกครั้ง
Target toks : เขา ปัด มัน อีกครั้ง

Prediction 1: เขา ถอด มัน ออก อีก </s>
Prediction 2: เขา ถอด มัน ออก </s>
Prediction 3: เขา ถอด มัน ออก อีกแล้ว </s>
Prediction 4: เขา ออก ไป อีก </s>
Prediction 5: เขา ปิด มัน </s>



2
text   : He brushed it off.
tokens : ['He', 'brushed', 'it', 'off', '.']

Target      : เขาปัดมัน
Target toks : เขา ปัด มัน

Prediction 1: โช </s>
Prediction 2: ฉัน อยาก ให้ คุณ ช่วย </s>
Prediction 3: ฉัน อยาก ให้ เธอ ช่วย </s>
Prediction 4: ฉัน ไม่รู้ </s>
Prediction 5: ฉัน รู้ </s>



3
text   : He buried his face again in his hands and groaned aloud.
tokens : ['He', 'buried', 'his', 'face', 'again', 'in', 'his', 'hands', 'and', 'groaned', 'aloud', '.']

Target      : เขาฝังใบหน้าของเขาอีกครั้งในมือของเขาแล้วส่งเสียงครวญคราง
Target toks : เขา ฝัง ใบหน้า ของ เขา อีกครั้ง ใน มือ ของ เขา แล้ว ส่งเสียง ครวญคราง


Target      : เขาเข้ามาแล้วพูดกับคนเฝ้าประตู
Target toks : เขา เข้ามา แล้ว พูด กับ คน เฝ้า ประตู

Prediction 1: เขา เข้ามา คุย กับ พอ ร์ เต อร ์ </s>
Prediction 2: เขา เข้ามา และ คุย กับ พอ ร์ เต อร ์ </s>
Prediction 3: เขา เข้ามา และ พูด กับ พอ ร์ เต อร ์ </s>
Prediction 4: เขา เข้ามา แล้ว คุย กับ พอ ร์ เต อร ์ </s>
Prediction 5: เขา เข้ามา คุย กับ พอ ท เต อร ์ </s>



19
text   : He came in and stood breathing asthmatically.
tokens : ['He', 'came', 'in', 'and', 'stood', 'breathing', 'asthmatically', '.']

Target      : เขาเข้ามาแล้วยืนหายใจหอบ
Target toks : เขา เข้ามา แล้ว ยืน หายใจ หอบ

Prediction 1: เขา เข้ามา และ เข้ามา และ ยืนยัน การ หายใจ </s>
Prediction 2: เขา เข้ามา และ เข้ามา และ ยืนยัน ว่า </s>
Prediction 3: เขา เข้ามา และ เข้ามา </s>
Prediction 4: เขา เข้ามา และ ยืนยัน ว่า </s>
Prediction 5: เขา เข้ามา และ เข้ามา และ ยืนยัน </s>



20
text   : He came into the room with an air of importance.
tokens : ['He', 'came', 'into', 'the', 'room', 'with', 'an', 'air', 'of', 'importa

KeyboardInterrupt: 

In [25]:
PATH_MODEL_TH_EN = './data/opensubtitles_model/exp003-1/transformer_base/checkpoint3.pt'

print('| loading model from {}'.format(PATH_MODEL_TH_EN))
state = checkpoint_utils.load_checkpoint_to_cpu(PATH_MODEL_TH_EN)
args = vars(state['args'])
ensemble_th_en, _, _ = checkpoint_utils.load_model_ensemble_and_task([PATH_MODEL_TH_EN], arg_overrides=args, task=None)


th_dict_2 = data.Dictionary()
th_dict_2.add_from_file('./data/opensubtitles_bin_word_en-th/dict.th.txt')

en_dict_2 = data.Dictionary()
en_dict_2.add_from_file('./data/opensubtitles_bin_word_en-th/dict.en.txt')


| loading model from ./data/opensubtitles_model/exp003-1/transformer_base/checkpoint3.pt
| [th] dictionary: 88848 types
| [en] dictionary: 90000 types


In [26]:

BEAM_SIZE = 5

_generator = SequenceGenerator(tgt_dict=en_dict_2, beam_size=BEAM_SIZE)

print('MT TH to EN, Word to Word')

for i, src in enumerate(texts['th'][:300]):
    print(i+1)
    print('text   :',src)
    toks = word_tokenize(src, engine='newmm', keep_whitespace=False)
    print('tokens :', toks)
    src_indices = [th_dict_2.index(t) for t in toks]
    src_len = [len(src_indices)]
  
    src_indices_tensor = torch.LongTensor(src_indices).view(1, -1)
    src_len_tensor = torch.LongTensor(src_len).view(1, -1)
    
    print('src indices:', src_indices_tensor.size(), src_indices_tensor)
    print('src len    :', src_len_tensor.size(), src_len_tensor)

    sample = {
        'net_input': {
                'src_tokens': src_indices_tensor, 'src_lengths': src_len_tensor,
        },
    }
    
    hypos = _generator.generate(models=ensemble_th_en, sample=sample)
    
    print('\nTarget      :', texts['en'][i])
    print('Target toks : {}'.format(' '.join(word_tokenize(texts['en'][i], keep_whitespace=""))))
    print('')
    for i in range(0, BEAM_SIZE):
        hypo = hypos[0][i]
        indices = list(hypo['tokens'].clone().detach().numpy())
        
        out_symbols = ' '.join([en_dict_2[idx] for idx in indices])
        print('Prediction {}: {}'.format(i+1, out_symbols))
        
    print('\n\n')
#     break

MT TH to EN, Word to Word
1
text   : เขาปัดมันอีกครั้ง
tokens : ['เขา', 'ปัด', 'มัน', 'อีกครั้ง']
src indices: torch.Size([1, 4]) tensor([[  19, 6285,    9,  383]])
src len    : torch.Size([1, 1]) tensor([[4]])

Target      : He brushed it off once more.
Target toks : He brushed it off once more .

Prediction 1: That ' s it . </s>
Prediction 2: He ' s got it . </s>
Prediction 3: That ' s what he said . </s>
Prediction 4: He ' s got it ! </s>
Prediction 5: That ' s it ! </s>



2
text   : เขาปัดมัน
tokens : ['เขา', 'ปัด', 'มัน']
src indices: torch.Size([1, 3]) tensor([[  19, 6285,    9]])
src len    : torch.Size([1, 1]) tensor([[3]])

Target      : He brushed it off.
Target toks : He brushed it off .

Prediction 1: It ' s okay . </s>
Prediction 2: It ' s ... </s>
Prediction 3: It ' s not . </s>
Prediction 4: - Yeah . </s>
Prediction 5: What ? </s>



3
text   : เขาฝังใบหน้าของเขาอีกครั้งในมือของเขาแล้วส่งเสียงครวญคราง
tokens : ['เขา', 'ฝัง', 'ใบหน้า', 'ของ', 'เขา', 'อีกครั้ง', 'ใน', 'มื


Target      : He came first a year ago, and revealed himself to Jeanne.
Target toks : He came first a year ago , and revealed himself to Jeanne .

Prediction 1: He came here a year ago . </s>
Prediction 2: He ' s here . </s>
Prediction 3: He came here . </s>
Prediction 4: He came here a year . </s>
Prediction 5: He came over here . </s>



17
text   : เขากลับมาบ้านใต้เสื้ออันเปียกชุ่ม
tokens : ['เขา', 'กลับมา', 'บ้าน', 'ใต้', 'เสื้อ', 'อัน', 'เปียกชุ่ม']
src indices: torch.Size([1, 7]) tensor([[   19,   184,   161,   852,   898,   365, 27678]])
src len    : torch.Size([1, 1]) tensor([[7]])

Target      : He came home drenched in his T-Shirt.
Target toks : He came home drenched in his T-Shirt .

Prediction 1: It ' s a house . </s>
Prediction 2: That ' s right . </s>
Prediction 3: It ' s the house . </s>
Prediction 4: The house . </s>
Prediction 5: It ' s a house </s>



18
text   : เขาเข้ามาแล้วพูดกับคนเฝ้าประตู
tokens : ['เขา', 'เข้ามา', 'แล้ว', 'พูด', 'กับ', 'คน', 'เฝ้า', 'ประตู']
sr


Target      : He carried in his hand a piece of thin, strong whipcord.
Target toks : He carried in his hand a piece of thin , strong whipcord .

Prediction 1: He ' s carrying a lot of them . </s>
Prediction 2: He ' s got a lot of them . </s>
Prediction 3: He ' s got a lot of hair . </s>
Prediction 4: He ' s got a little bit of a lot of wood . </s>
Prediction 5: He ' s carrying a little bit of a lot . </s>



32
text   : เขาถือมันไปต้องแสงที่สาดเข้ามาจากหน้าต่าง
tokens : ['เขา', 'ถือ', 'มัน', 'ไป', 'ต้อง', 'แสง', 'ที่', 'สาด', 'เข้า', 'มาจาก', 'หน้าต่าง']
src indices: torch.Size([1, 11]) tensor([[  19,  855,    9,   10,   36, 1139,    6, 5527,  177,  250, 1348]])
src len    : torch.Size([1, 1]) tensor([[11]])

Target      : He carried it over to the light of the open window.
Target toks : He carried it over to the light of the open window .

Prediction 1: He ' s got to go . </s>
Prediction 2: He ' s got to be in there . </s>
Prediction 3: He ' s got it . </s>
Prediction 4: That ' s it 


Target      : He closely resembled Elvis.
Target toks : He closely resembled Elvis .

Prediction 1: He ' s got a face . </s>
Prediction 2: He ' s in the face . </s>
Prediction 3: He ' s got the face . </s>
Prediction 4: He looks like he . </s>
Prediction 5: He looks like he ' s a face . </s>



48
text   : เขามุงหลังคา
tokens : ['เขา', 'มุง', 'หลังคา']
src indices: torch.Size([1, 3]) tensor([[  19, 8880, 2552]])
src len    : torch.Size([1, 1]) tensor([[3]])

Target      : He clutched the thatch.
Target toks : He clutched the thatch .

Prediction 1: It ' s okay . </s>
Prediction 2: It ' s ... </s>
Prediction 3: You ' re not . </s>
Prediction 4: What ? </s>
Prediction 5: I ... </s>



49
text   : เขาเทียบมันกับลายมือบนบัตร
tokens : ['เขา', 'เทียบ', 'มัน', 'กับ', 'ลาย', 'มือบน', 'บัตร']
src indices: torch.Size([1, 7]) tensor([[   19,  2342,     9,    31,  2806, 17981,  1868]])
src len    : torch.Size([1, 1]) tensor([[7]])

Target      : He compared it with the handwriting upon the card.



Target      : He crept on tiptoe to his side.
Target toks : He crept on tiptoe to his side .

Prediction 1: He ' s hanging out . </s>
Prediction 2: He ' s hanging out with his toes . </s>
Prediction 3: He ' s hanging out with his feet . </s>
Prediction 4: He ' s coming up . </s>
Prediction 5: He ' s coming . </s>



63
text   : เขาร้องด้วยความไม่พอใจอย่างมากจนเธออดหัวเราะไม่ได้
tokens : ['เขา', 'ร้อง', 'ด้วย', 'ความไม่พอใจ', 'อย่างมาก', 'จน', 'เธอ', 'อด', 'หัวเราะ', 'ไม่ได้']
src indices: torch.Size([1, 10]) tensor([[   19,   884,    53, 11009,  1661,   449,    13,  2807,  1053,    43]])
src len    : torch.Size([1, 1]) tensor([[10]])

Target      : He cried in such genuine dismay that she broke into hearty laughter.
Target toks : He cried in such genuine dismay that she broke into hearty laughter .

Prediction 1: He cried so hard that she couldn ' t . </s>
Prediction 2: He cried so much that she couldn ' t . </s>
Prediction 3: He cried so bad that she couldn ' t . </s>
Prediction 4: H

...


Target      : He was fond of quoting a fragment from a certain poem.
Target toks : He was fond of quoting a fragment from a certain poem .

Prediction 1: He ' s talking . </s>
Prediction 2: He ' s talking about it . </s>
Prediction 3: He ' s got a poem . </s>
Prediction 4: He ' s got it . </s>
Prediction 5: He speaks . </s>



227
text   : เขาขมวดคิ้วอย่างครุ่นคิด
tokens : ['เขา', 'ขมวดคิ้ว', 'อย่าง', 'ครุ่นคิด']
src indices: torch.Size([1, 4]) tensor([[   19, 33706,   117, 12205]])
src len    : torch.Size([1, 1]) tensor([[4]])

Target      : He was frowning thoughtfully.
Target toks : He was frowning thoughtfully .

Prediction 1: What do you mean ? </s>
Prediction 2: That ' s good . </s>
Prediction 3: That ' s right . </s>
Prediction 4: That ' s all . </s>
Prediction 5: What ? </s>



228
text   : เขาชอบการอ้างอิงประโยคจากบทกวี
tokens : ['เขา', 'ชอบ', 'การ', 'อ้างอิง', 'ประโยค', 'จาก', 'บทกวี']
src indices: torch.Size([1, 7]) tensor([[  19,  144,   61, 5694, 3392,   69, 4820]])
src l


Target      : He was not allowed, however, to reflect upon these remarkable problems.
Target toks : He was not allowed , however , to reflect upon these remarkable problems .

Prediction 1: He ' s not allowed . </s>
Prediction 2: He ' s not allowed any of this . </s>
Prediction 3: He ' s not allowed to do this . </s>
Prediction 4: He wasn ' t allowed . </s>
Prediction 5: He wasn ' t allowed to do this . </s>



243
text   : เขาไม่ได้อยู่บ้านเมื่อคืน
tokens : ['เขา', 'ไม่ได้', 'อยู่', 'บ้าน', 'เมื่อคืน']
src indices: torch.Size([1, 5]) tensor([[ 19,  43,  27, 161, 945]])
src len    : torch.Size([1, 1]) tensor([[5]])

Target      : He was not at the villa last night.
Target toks : He was not at the villa last night .

Prediction 1: He ' s not here . </s>
Prediction 2: He ' s not . </s>
Prediction 3: It ' s not . </s>
Prediction 4: He wasn ' t . </s>
Prediction 5: Not here . </s>



244
text   : เขาไม่ได้ถูกทำให้ไม่พอใจด้วยความกระตือรือร้นของเพื่อนของเขา
tokens : ['เขา', 'ไม่ได้', 'ถูก',


Target      : He was sheer brain, icy, cool, calculating, as ruthless as a steam hammer.
Target toks : He was sheer brain , icy , cool , calculating , as ruthless as a steam hammer .

Prediction 1: He ' s smart . </s>
Prediction 2: He ' s very smart . </s>
Prediction 3: He was smart . </s>
Prediction 4: He ' s pretty smart . </s>
Prediction 5: He ' s smart , he ' s mean . </s>



259
text   : \nเขานั่งตัวตรงบนเก้าอี้ของเธอรุนแรงเกือบโกรธเคืองด้วยความโกรธ
tokens : ['\\', 'n', 'เขา', 'นั่ง', 'ตัวตรง', 'บน', 'เก้าอี้', 'ของ', 'เธอ', 'รุนแรง', 'เกือบ', 'โกรธเคือง', 'ด้วย', 'ความโกรธ']
src indices: torch.Size([1, 14]) tensor([[  182,  3389,    19,   347, 14991,   187,  2240,    14,    13,  1818,
           814, 16468,    53,  3234]])
src len    : torch.Size([1, 1]) tensor([[14]])

Target      : He was sitting erect in her chair, violent, almost rancorous with anger.
Target toks : He was sitting erect in her chair , violent , almost rancorous with anger .

Prediction 1: He ' s sitting on he


Target      : He was surprised; but it was natural, wasn't it?
Target toks : He was surprised ; but it was natural , wasn ' t it ?

Prediction 1: He feels like he ' s a person , isn ' t he ? </s>
Prediction 2: He feels like he ' s a regular person . </s>
Prediction 3: He feels like he ' s a normal person . </s>
Prediction 4: He ' s not ? </s>
Prediction 5: He feels like he ' s a person . </s>



272
text   : เขามีความสัมพันธ์กับพวกเขาทั้งคู่
tokens : ['เขา', 'มี', 'ความสัมพันธ์', 'กับ', 'พวกเขา', 'ทั้งคู่']
src indices: torch.Size([1, 6]) tensor([[  19,   28, 1028,   31,   70,  797]])
src len    : torch.Size([1, 1]) tensor([[6]])

Target      : He was the associate of them both.
Target toks : He was the associate of them both .

Prediction 1: He ' s having an affair with them . </s>
Prediction 2: He had an affair with them . </s>
Prediction 3: He was having an affair with them . </s>
Prediction 4: He ' s having a relationship with them . </s>
Prediction 5: He ' s having a connection w


Target      : He was unfairly classified as an economic migrant.
Target toks : He was unfairly classified as an economic migrant .

Prediction 1: It ' s all right . </s>
Prediction 2: It ' s a deal . </s>
Prediction 3: It ' s a ... </s>
Prediction 4: It ' s not . </s>
Prediction 5: That ' s what I meant . </s>



287
text   : เขาเป็นคนไร้เหตุผล ชักชวนไม่ได้ และใช้ภาษามากเกินควร
tokens : ['เขา', 'เป็น', 'คน', 'ไร้เหตุผล', 'ชักชวน', 'ไม่ได้', 'และ', 'ใช้', 'ภาษา', 'มาก', 'เกินควร']
src indices: torch.Size([1, 11]) tensor([[   19,    20,    38,  8247, 10683,    43,    30,   142,  1244,    67,
         19688]])
src len    : torch.Size([1, 1]) tensor([[11]])

Target      : He was unreasonable and unpersuadable and used intemperate language.
Target toks : He was unreasonable and unpersuadable and used intemperate language .

Prediction 1: He can ' t be used to use it . </s>
Prediction 2: He can ' t be trusted . </s>
Prediction 3: He can ' t be used to it . </s>
Prediction 4: He can ' t be u

In [286]:
# from fairseq import checkpoint_utils, data, options, tasks

# # Parse command-line arguments for generation
# # parser = options.get_parser(default_task='translation')
# # print(parser)
# # args = options.parse_args_and_arch(parser)

# # print(args)
# # # Setup task
# # task = tasks.get_task(args)

# PATH = './data/opensubtitles_model/exp002-1/transformer_base_revised/checkpoint_best.pt'


# print('| loading model from {}'.format(PATH))
# state = checkpoint_utils.load_checkpoint_to_cpu(PATH)
# args = vars(state['args'])
# print(args)


# ensemble, args, _task = checkpoint_utils.load_model_ensemble_and_task([PATH], arg_overrides=args, task=None)

# # print(task)

In [220]:
# ensemble[0].eval()


In [223]:
# ensemble[0].training


In [224]:

# print(th_dict.index('ฉัน'))
# print(en_dict.index('A'))

# print('index of <eos> in th dict', th_dict.eos())
# print('index of <eos> in en dict', en_dict.eos())

In [225]:
# print(ensemble)

In [222]:
# # example EN -> TH

# en_sents = [
#     'I love you.'
# ]
# for src in en_sents:
#     print('text   :',src)
#     toks = word_tokenize(src, engine='newmm', keep_whitespace=False)
#     print('tokens :', toks)
#     src_indices = [en_dict.index(t) for t in toks]
#     src_len = len(src_indices)
  
#     src_indices_tensor = torch.tensor(src_indices).view(1, -1)
#     src_len_tensor = torch.tensor(src_len).view(1)
    
#     print('src indices:', src_indices_tensor.size(), src_indices_tensor)
#     print('src len    :', src_len_tensor.size(), src_len_tensor)

#     encoder_outputs = ensemble[0].encoder.forward(src_indices_tensor, src_len_tensor)
#     #  (src_len, batch, features)
#     print('encoder_outputs', encoder_outputs)
    
    
#     prev_tok = torch.tensor([[[th_dict.bos()]]]).view(1, 1)
#     print('prev_tok th <bos>', prev_tok)
    
    
#     for i in range(40):
#         decoder_output, _dict = ensemble[0].decoder.forward(prev_tok, encoder_outputs)
        
#         print('decoder_output', decoder_output.size())
#         index_top = decoder_output.topk(1)[1]
        
#         prev_tok = torch.tensor(index_top).view(1, 1)
#         print('prev_tok', prev_tok)
        
#         th_index = prev_tok[0][0].clone().detach().numpy()
#         if th_index == th_dict.eos():
#             break
#         print('TH {} = {}'.format(i, th_dict[th_index]))
# # #         if prev_tok.view(1) == th_dict.eos():
# # #             break
# #     # batch_size, tgt_len, vocab
# #     print('end')
# #     break


In [195]:
# ensemble[0].forward(src_trg['en'][0], len(src_trg['en'][0]), [0])